## Topic Model - LDA using Collapsed Gibbs Sampling
Sandeep Shetty  
12/10/2018

**Latent Dirichlet Allocation (LDA)** is a probabilistic generative topic modeling approach (Blei et al, 2002). Given a distribution of words contained in a document or set of documents, LDA infers the latent (or hidden) distribution of topics within a document and/or a corpus of documents. 

In LDA, one sets up a generative model of document formation, where each document is a mixture of topics, and each topic is a distribution over words. Using Bayesian principles, LDA allows one to infer the posterior distribution of topic given the observed document/corpus, and Bayesian networks to exploit the conditional dependencies in the hierarchical model to make the computation of the posterior distribution feasible. 

Given the complex nature of the posterior distribution, the estimation is achieved through simulation procedures. There are several different approaches  in the literature, each with its pros and cons. Blei et al (2002) use a variational EM algorithm (VI). VI is fast but not accurate.  Griffith & Steyvers (2005) use MCMC-technique of Collapsed Gibbs Sampling (CGS). CGS is "more" precise and popular among topic model researchers but it is inefficient on large samples (Blei et al 2018).

In this exercise, I used **Collapsed Gibbs Sampling (CGS)** to approximate the posterior distribution of hidden variables, i.e. topic assignment. This distribution is subsequently used to estimate the key parameters of interest - the distribution of topics within each documnent in the corpus and the distribution of topics over vocabulory. The number of topics to search in a corpus is an provided by the user. 

The posterior distribution of latent variable, i.e. topic assignment, that is approximated using CGS is given as:  
  
$${
P( z_i = j \text{ }| \text{ } z_{-i}, w_i, m_i ) 
    \propto \frac{ n^{(t)}_{k,-i} + \beta }{ \sum^V_{ t = 1 }n^{(t)}_{k,-i} + V\beta } \times
      \frac{ n^{(t)}_{m,-i} + \alpha }{ \sum^K_{ k = 1 }n^{(k)}_{m} + K\alpha - 1 }}$$

where,

$P( z_i = j \text{ }| \text{ } z_{-i}, w_i, m_i ) $ : The probability that token $i$ in document $d$ is assigned to topic $j$ conditioned (or knowing) the topic assignments of all other tokens in document $d$ and the observed word $w$ at token $i$     
$z_{−i}$ : Topic assignments of all other tokens;     
$w_{i}$ : Word (index) of the ith token;      
$m_{i}$ : Document containing the ith token.  

This probability is proportional to the right side:-  
$V$: Total number of words in the corpus   
$k$: one of the topics, $K$ is total number of topics   
$n^{(t)}_{k,-i}$ : number of words, excluding $i$, which are assigned topic $k$ in document $m$   
$n^{(t)}_{m,-i}$ : number of words, excluding $i$, in document $m$ those are assigned topic $k$  
$\sum^V_{ t = 1 }n^{(t)}_{k,-i}$: number of words,excluding $i$, are assigned topic $k$  
$\sum^K_{ k = 1 }n^{(k)}_{m}$: length of document $m$  
$\beta$ : Parameter that sets the topic distribution for the words.  
$\alpha$ : Parameter that sets the topic distribution for the documents.  

The right hand side is basically a product of two proportions, which balances the within document occurrence of a given topic with that observed and across documents in a corpus. 

The parameters that are computed once the full conditional distribution of topic assignment is obtained via CGS are:

$$\phi_{k,t} = \frac{n^{(t)}_{k} + \beta}{\sum^V_{ t = 1 }n^{(t}_{k} + V\beta}$$

$$\theta_{m,k} = \frac{n^{k}_{m} + \alpha}{\sum^K_{ k = 1 }n^{(k)}_{m} + K\alpha}$$

$\phi_{k,t}$ is the probability of word $t$ for topic $k$, and $\theta_{m,k}$ is the proportion of topic $k$ in document $m$. 

Since CGS is quite popular among the topic model research community, in the literature there are several alterations proposed to CGS to improve its performance on large sample. However, the issue that gathers very scant mention in the topic modeling community but of importance to many applied Bayesian researchers is the discussion of convergence of CGS - whether the starting distribution converges to the target distribution. Although, there are quite a few in the literature on MCMC but these methods rarely agree with each other (Cowles & Carlin (1996). In the topic modelling work, I noticed the use a different measure to assess convergence, than called `Perplexity`, which is basically measuring on how "well" the model is clustering semantically related terms in a corpus. 

In the code below, I have not included any convergence diagnostics to test if the simulated posterior is reaching the target distribution. But I do account for multiple starting points of the distribution in the code. This practice helps to traverse the distribution from different vantage points. However, I do plan to revisit the interesting issue of MCMC convergence disgnostics in the near future.  

The LDA model used is as described in Blei et al (2002). There are several other extensions of the LDA and other  topic model tools as described in Minh's topic model protocol document that would be interesting to explore and build from here.

Since the code is using the collapsed Gibbs Sampler to simulate, it can be adapted to a parallelized environment for faster computation. 

**Next steps**: 
- Parallelization to improve efficiency
- Reporting and visualization of output
- Writing a more modular (cleaner) code
- Research alternate preprocessing techniques
- Implement other extended LDA models
- Implement Variational Inference technique for parameter estimation (Blei et al)
- Matrix-Decomposition methods for Topic Modeling (Minh protocol)
- Investigate and implement  convergence diagnotics

### Modules

In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
import nltk

In [4]:
from nltk.tokenize import RegexpTokenizer # remove stop words
tokenizer = RegexpTokenizer(r'\w+')

In [5]:
nltk.download('stopwords')    

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/getsane/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
from nltk.corpus import stopwords

### Data import

In [7]:
doc = pd.read_csv("tag-data.csv", header=[1])
doc = doc.drop(index=0).reset_index()

In [8]:
doc = doc[:1000]

In [9]:
docli = doc.comments.tolist() #Each item in this list is a separate comment/document"

### Text cleaning
A. Remove Stop words  
B. Create dictionary of document composed of terms and frequency

In [10]:
stop_words = stopwords.words('english')

In [11]:
tkn_doc = []

In [12]:
for i,k in enumerate(docli):
    wrlist = tokenizer.tokenize(k.lower()) 
    tkn_doc.append([k for k in wrlist if not k in stop_words])

In [13]:
doct = dict()
for i,k in enumerate(tkn_doc):
    cnt = dict() 
    for w in k:
        if w in cnt.keys():
            cnt[w] += 1
        else:
            cnt[w] = 1
    doct[i] = cnt

### Creating a term-document matrix
A. Create a list of terms across the whole corpus  
B. Remove alphanumeric terms

In [14]:
# create a sparse matrix of term-document
terms = []
for i in range(len(doct)):
    for j in doct[i].keys():
        terms.append(j)
        
terms = list(set(terms))

In [15]:
# Remove digits and digit combined words

p = re.compile(r'\d+')
termClean = [terms[i] for i in range(len(terms)) 
             if p.findall(terms[i])==[]] 

In [16]:
# Obtain document count and term count
len_t = len(termClean)
len_d = len(doct)

In [17]:
term_doc = np.empty((len_t,len_d), dtype=np.int)

In [18]:
# Term-Document Frequency Matrix 
# Columns-->Document; Rows --> Terms
# E.g. term_doc[:,0] -- Extracting 1st document

for i in range(len_d):
    for t,word in enumerate(termClean):
        if word in doct[i].keys():
            term_doc[t,i]=doct[i].get(word)
        else:
            term_doc[t,i] = 0

In [19]:
# Think of using Scipy's COO later to save memory for large dataset

### Collapsed Gibbs Sampling 
A. Generate conditional topic assignment distribution $P( z_i = j \text{ }| \text{ } z_{-i}, w_i, d_i ) $  
B. Multiple iterations from different starting points of the starting distributions  
C. Remove burn-in sample from each different phase
D. Collect the different phases and sample to calculate the final parameters

In [20]:
# Initialization of CGS

def cgsIntialization(k, wordDocMat):
    
    '''
    Input: 
        k = # of topic (int)
        chain = # of different starting point for the iterations (int)
        wordDocMat = Word-document matrix -- [V X d] array
    
    Output:
        n_t_k - number of times term 't' in topic 'k' - [V X k] array
        n_k_m - number of times topic 'k' in document 'd' - [d X k] array
        n_k - number of words in each topic
        n_m - length of document
        k - number of topics 
    
    '''
    
    len_t = wordDocMat.shape[0]  # number of words
    len_d = wordDocMat.shape[1]  # number of docs
    mat_V_k = np.empty((len_d,len_t,k), dtype = np.int)  #shape==>(doc, words, topic)
    
    for doc in range(len_d):
        for index, val in enumerate(wordDocMat[:,doc]):
            if  val!= 0:
                mat_V_k[doc,index,:] = np.random.multinomial(1,[1./k]*k) 
            else:
                mat_V_k[doc,index,:] = 0    
    return(mat_V_k)

In [21]:
def cgsIteration(wordDocMat, niter = 1200 , burn = 200, k = 5, 
                 alpha = 0, beta = 0):
    
    '''
    Defintion: Iteration of the CGS 
    
    Input: 
    
        Output returned from cgsIntialization
        niter - total number of iterations
        burn -  number of iterations to discard
        alpha - hyperparameter influencing the shape of the Dirichlet priors
        beta -  " 
        wordDocMat - term-document matrix (V x d)
    
    Output: 
    
        finalThetaMK - array [d X k] - distribution of topic in a document
                                       (averaged over multiple iterations)
        finalphiTK -  array [t X k ] - distribution over word for each topic
                                       (averaged over multiple iterations) 
        mat_V_k - [d X t X k] - topic assignment for each for all document
    
    '''
    
    intIter = 0
    avgSmp = niter-burn

    mat_V_k = cgsIntialization(k, wordDocMat)
     
    n_k_m = np.sum(mat_V_k, axis=1)  #{doc, topics}
    n_t_k = np.sum(mat_V_k, axis=0)  #{words,topics}
    n_k = np.sum(n_t_k, axis=0)      #{words}
    n_m = np.sum(n_k_m, axis=1)      #{topics}
    
    lenM = mat_V_k.shape[0]
    lenT = mat_V_k.shape[1]
     
    thetaMK = np.empty((lenM,k,avgSmp), dtype = np.float) 
    phiTK = np.empty((lenT,k,avgSmp), dtype = np.float)
     
    while intIter < niter:
        
        for doc in range(len_d):   # each document
            for index, val in enumerate(wordDocMat[:,doc]): # each word
                if  val != 0:   
                    
                    # check if word is in ...
                    # subtracting its count from the ...
                    # topic assigned to the word in initialization;
                    
                    n_k_m[doc,:] = n_k_m[doc,:] - mat_V_k[doc,index,:]
                    n_t_k[index,:] = n_t_k[index,:]-mat_V_k[doc,index,:]
                    n_k = n_k - mat_V_k[doc,index,:]
                    n_m[doc] = n_m[doc] - 1 

                    prob = np.empty(k, dtype=np.float)

                    for topic in range(k):
                        rhst1 = (n_k_m[doc,topic] + alpha)/((n_m[doc] + k*alpha)-1) 
                        rhst2 = (n_t_k[index,topic] + beta)/(n_k[topic] + lenT*beta)

                        prob[topic] = rhst1*rhst2

                    prob_new = prob/(np.sum(prob))

                    mat_V_k[doc,index,:] = np.random.multinomial(1,prob_new) 

                    n_k_m[doc,:] = n_k_m[doc,:] + mat_V_k[doc,index,:]                                                    
                    n_t_k[index,:] = n_t_k[index,:] + mat_V_k[doc,index,:]
                    n_k = n_k + mat_V_k[doc,index,:]
                    n_m[doc] = n_m[doc] + 1
        
        #print(intIter)
        
        # calculate the theta & phi parameters     
        if intIter >= burn:
            n_k_m1 = np.sum(mat_V_k, axis=1)  #{doc, topics}
            print(intIter)
            n_t_k1 = np.sum(mat_V_k, axis=0)  #{words,topics}
            n_k1 = np.sum(n_t_k, axis=0)      # words count by topic
            n_m1 = np.sum(n_k_m, axis=1)      # length of document
            
            dim3 = intIter - burn
            
            for it1 in range(lenM):
                thetaMK[it1,:, dim3] = (n_k_m1[it1,:] + alpha)/(n_k_m1[it1,:].sum(axis=0) + k*alpha)
        
            for it2 in range(k):
                phiTK[:,it2, dim3] =  (n_t_k1[:,it2] + beta)/(n_t_k1[:,it2].sum(axis=0) + lenT*beta)
                
        intIter += 1
        
    finalThetaMK = np.mean(thetaMK, axis=2)
    finalphiTK = np.mean(phiTK, axis=2)
            
    return((finalThetaMK,finalphiTK,mat_V_k))          

### Setting up the LDA-CGS execution

In [27]:
def execCGS(chain):
    '''
    Input:
    
    chain - number of different starting point 
            covering the starting distributions. 
            Recommended practise in CGS sampling.
    Output:
    
    Output from cgsIteration
    
    finalThetaMK - array [d X k] - distribution of topic in a document
                                  (averaged over multiple iterations within one chain)
    finalphiTK -  array [t X k ] - distribution over word for each topic
                                  (averaged over multiple iterations) 
    mat_V_k - [d X t X k] - topic assignment for each for all document
    
    '''
    chainIter = {} #to store output from each chain separately
    
    for i in range(chain):
        chainIter[i] = cgsIteration(niter = 4, 
                                    burn = 2, 
                                    k = 5, 
                                    alpha = 50/5, #(=50/k)
                                    beta = 0.01, 
                                    wordDocMat=term_doc)
    
    return(chainIter)

### Running the CGS

In [ ]:
chain_out = execCGS(chain=1)

### Simple reporting

In [137]:
for i in range(5):
    strin1 = 'Topic'+str(i)
    data= pd.DataFrame(np.array(termClean),chain_out[0][1][:,i], columns=[strin1])
    print(data.sort_index(ascending=False).iloc[10:20])
    print("------------------")

              Topic0
0.006163         rin
0.006121      earned
0.006055       labor
0.005914      server
0.005811          go
0.005807        urge
0.005403     minimum
0.005335  regulation
0.005236         way
0.005139  restaurant
------------------
            Topic1
0.005677  proposed
0.005623  industry
0.005351       pay
0.005299       one
0.005244      hour
0.004980   servers
0.004869   minimum
0.004869      well
0.004652      fair
0.004596      paid
------------------
               Topic2
0.006411        staff
0.006029    employees
0.005920       owners
0.005920          pay
0.005813       server
0.005652  restaurants
0.005538          get
0.005433         rule
0.005378         back
0.005328         make
------------------
           Topic3
0.006225     many
0.006173     give
0.005856  workers
0.005751    house
0.005751     know
0.005751    would
0.005488    staff
0.005488      pay
0.005330  receive
0.005277     food
------------------
              Topic4
0.006502       money
0.

### *`{Side note}` *
**N-dimensional array**  
- A is a 3-D array (two stacked 2D arrays)
    - shape of A [documents,words,topics]
    - For e.g. shape [1000,5906,5]
    - Documents (1000), words (5906), topics (5)
        - E.g. 3-D array:-  
```c = np.array( [[[  0,  1,  2], [ 10, 12, 13]],
    ... [[100,101,102], [110,112,113]]])```
                              
    
- Sum along axis = 0 --> integrating over documents --> results in an array (word, topic) {summed over all documents}
- Sum along axis = 1 --> integrating over words --> resulting array is (doc, topic) {summed over all words}